In [1]:
# Library imports
import cadquery as cq

In [2]:
# Set variables
height=50
width= 200
length= 100
thickness = 3
rounded_edges= True
screw_radius=1

In [3]:
# Create top base
top_base=cq.Workplane().rect(width, length).extrude(thickness)
top_base

In [4]:
# Generate bottom of base
bottom_base=cq.Workplane().rect(width, length).extrude(height)
inner_base=cq.Workplane().rect(width-(thickness*2), length-(thickness*2)).extrude(height-thickness)
bottom_base=bottom_base.cut(inner_base.translate((0,0, thickness)))
bottom_base

In [5]:
# Add rounded edges
if rounded_edges:
    bottom_base=bottom_base.fillet((thickness/2)-0.01)
bottom_base

In [6]:
bottom_base=bottom_base.cut(top_base.translate((0,0, height-thickness)))
bottom_base

In [7]:
# Add rounded edges
if rounded_edges:
    top_base = top_base.faces(">Z").edges().fillet(thickness/2)
top_base

In [8]:
# Add corners for screw/plugs
corner_radius=thickness
corner=cq.Workplane().circle(corner_radius).extrude(height - (thickness*2))
positions = [
    (-(width/2 - corner_radius), -(length / 2 - corner_radius)),
    (-(width / 2 - corner_radius), length / 2 - corner_radius),
    (width / 2 - corner_radius, -(length / 2 - corner_radius)),
    (width / 2 - corner_radius, length / 2 - corner_radius),
    (0, -(length / 2 - corner_radius)),
    (0, length / 2 - corner_radius)
]
corners = cq.Workplane()
for pos in positions:
    corners = corners.union(corner.translate(pos).translate((0,0,thickness)))
bottom_base=bottom_base.union(corners)
bottom_base

In [10]:
# Add the screw holes or slots
corner_holes = cq.Workplane()
screw_holes = cq.Workplane()
if screw_radius > 0.0:
    corner_hole = cq.Workplane().circle(corner_radius / 2).extrude(thickness/2)
    screw_hole = cq.Workplane().circle(screw_radius).extrude(height)
else:
    corner_hole = cq.Workplane().circle(corner_radius / 2).extrude(thickness)
    screw_hole = cq.Workplane()
for pos in positions:
    corner_holes = corner_holes.union(corner_hole.translate(pos))
    if screw_radius > 0.0:
        screw_holes = screw_holes.union(screw_hole.translate(pos))
bottom_base = bottom_base.union(corner_holes.translate((0,0,height-thickness)))
if screw_radius > 0.0:
    bottom_base = bottom_base.cut(screw_holes.translate((0,0,thickness)))
bottom_base

In [11]:
# Add the screw holes or slots
top_base = top_base.cut(corner_holes)
if screw_radius > 0.0:
    top_base = top_base.cut(screw_holes)
top_base

In [12]:
# Create an assembly of the full base
base = cq.Assembly().add(top_base).add(bottom_base, loc=cq.Location((0, length, 0)))
base

In [13]:
# export step
base.save("test_steps/base.step")